In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
# First, we get the 800 timepoints we are considering as a list, to use for indices given by cluster-based permutation later
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')
pp = 0 #only need one participant

# we can use aligned data

id_directory = os.path.join(data_directory, 'p%03d/' % pp)
pp_directory = os.path.join(id_directory, erps)
fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (pp, 'early_late', 'aligned'))
evoked = mne.read_evokeds(fname)
df = evoked[0].to_data_frame()
time = df['time'].tolist()

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)


In [3]:
# Load data for each condition
# Take the mean across electrodes of interest per participant
# transform data into array accepted for cluster-based permutation test in mne
# output: (n_participants, n_timepts) for each condition

root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#specify channels we need - FRN
channels = ['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']

    
#read in evoked object
tasknames = ['rot', 'rdm', 'mir']
conditionnames = ['early', 'late']


#aligned
evoked_list = []

for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], 'early_late', 'aligned'))
    evoked = mne.read_evokeds(fname)
    evoked = evoked[0]
    evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
    evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
    evoked_list.append(evoked)
    
aligned_flist = evoked_list

#rot and rdm and mir
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], tasknames[task]))
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
            evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
            evoked_list.append(evoked)
        if task == 0 and condition == 0:
            early_rot_flist = evoked_list
        elif task == 0 and condition == 1:
            late_rot_flist = evoked_list
        elif task == 1 and condition == 0:
            early_rdm_flist = evoked_list
        elif task == 1 and condition == 1:
            late_rdm_flist = evoked_list
        elif task == 2 and condition == 0:
            early_mir_flist = evoked_list
        elif task == 2 and condition == 1:
            late_mir_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\p022_early_late_aligned-ave.fif ...


Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\frn\p009_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p010\frn\p010_early_rot-ave.fif ...
    Found the data of inter

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\frn\p028_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\frn\p029_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\frn\p030_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF co

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p017\frn\p017_late_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p018\frn\p018_late_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified 

...

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\frn\p013_late_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\frn\p014_late_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\frn\p015_late_rdm-ave.fif ...
    Found the data of interest

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p003\frn\p003_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specifie

        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\p022_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p023\frn\p023_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data 

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p010\frn\p010_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p011\frn\p011_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRev

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\frn\p030_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\frn\p031_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)


In [4]:
# Comparing two ERP signals is just the same as taking their difference (ERP1 minus ERP2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in ERP plots

flists = [early_rot_flist, late_rot_flist, early_rdm_flist, late_rdm_flist, early_mir_flist, late_mir_flist]
conditionnames = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], aligned_flist, p, perms)
#     print(clust_idx, clust_pvals)
    for clust in range(0, len(clust_idx)):
        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
        cluster_start = cluster.start
        clust_idx_start.append(cluster_start)
        
        cluster_end = cluster.stop
        clust_idx_end.append(cluster_end)
        
        time_idx_start = time[cluster_start]
        time_start.append(time_idx_start)
        
        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
        time_end.append(time_idx_end)
        
        clust_p = clust_pvals[clust]
        p_values.append(clust_p)
        
        condition.append(conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_vsAligned_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

In [5]:
# Next, we subtract aligned from each condition, so that we can compare early vs late in each perturbation type
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
earlyrot_diff = []
laterot_diff = []
earlyrdm_diff = []
laterdm_diff = []
earlymir_diff = []
latemir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(early_rot_flist, aligned_flist)
        earlyrot_diff.append(diffevks)
        earlyrot_diff = earlyrot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(late_rot_flist, aligned_flist)
        laterot_diff.append(diffevks)
        laterot_diff = laterot_diff[0]
    elif cond == 2:
        diffevks = np.subtract(early_rdm_flist, aligned_flist)
        earlyrdm_diff.append(diffevks)
        earlyrdm_diff = earlyrdm_diff[0]
    elif cond == 3:
        diffevks = np.subtract(late_rdm_flist, aligned_flist)
        laterdm_diff.append(diffevks)
        laterdm_diff = laterdm_diff[0]
    elif cond == 4:
        diffevks = np.subtract(early_mir_flist, aligned_flist)
        earlymir_diff.append(diffevks)
        earlymir_diff = earlymir_diff[0]
    elif cond == 5:
        diffevks = np.subtract(late_mir_flist, aligned_flist)
        latemir_diff.append(diffevks)
        latemir_diff =latemir_diff[0]

In [ ]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of EARLY VS LATE comparisons
# This information can then be included in ERP difference wave plots

conds = ['rot_diff', 'rdm_diff', 'mir_diff']
conditionnames = ['rot', 'rdm', 'mir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(laterot_diff, earlyrot_diff, p, perms)
#         print(clust_idx, clust_pvals)
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[c])
            
    elif c == 1:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(laterdm_diff, earlyrdm_diff, p, perms)
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[c])
            
    elif c == 2:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(latemir_diff, earlymir_diff, p, perms)
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_EarlyvsLate_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

In [6]:
# Next step is to subtract early from late condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
rot_diff = []
rdm_diff = []
mir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(laterot_diff, earlyrot_diff)
        rot_diff.append(diffevks)
        rot_diff = rot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(laterdm_diff, earlyrdm_diff)
        rdm_diff.append(diffevks)
        rdm_diff = rdm_diff[0]
    elif cond == 2:
        diffevks = np.subtract(latemir_diff, earlymir_diff)
        mir_diff.append(diffevks)
        mir_diff = mir_diff[0]

In [9]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of PERTURBATION comparisons
# This information can then be included in ERP difference wave plots

conds = ['rotvmir', 'rotvrdm', 'mirvrdm']
conditionnames = ['rotvmir', 'rotvrdm', 'mirvrdm']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, mir_diff, p, perms)
#         print(clust_idx, clust_pvals)
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[c])
            
    elif c == 1:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, rdm_diff, p, perms)
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[c])
            
    elif c == 2:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(mir_diff, rdm_diff, p, perms)
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_PerturbTypeComp_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-1.916749 max=3.207027
Running initial clustering …
Found 12 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-3.088818 max=3.200277
Running initial clustering …
Found 13 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.468024 max=1.464930
Running initial clustering …
Found 3 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]